In [ ]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [ ]:
resource_name = "ml_rs"
workspace_name = "ml_ws"
subscriptionID = "*****" 
aml_compute_target = "ml-cluster" 
experiment_name= 'ml_expirement'

In [ ]:
#creating Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="Canada Central")
    ws.write_config('.azureml')


In [ ]:
Workspace.from_config()

In [ ]:
# Creating Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")


In [ ]:
# Creating Experiment
exp = Experiment(ws,experiment_name)

In [ ]:
# Create environment to execute your code
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")

config=ScriptRunConfig(source_directory="./",script="mytrain_log.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

In [ ]:
# Changing parameters
config=ScriptRunConfig(source_directory="./",script="mytrain_log.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7])
execution=exp.submit(config)

In [ ]:
#Registering model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./diabeticmodel.pkl",model_name="first_ml_model")

In [ ]:
model = Model(ws,"first_ml_model")

In [ ]:
# Register the environment
print("Registering the environment...")
env.register(ws)

In [ ]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py',source_directory='.',environment=env)

In [ ]:
from azureml.core.webservice import AksWebservice
print('Creating the Deployment configuration for webservice...')
aciconfig = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                   memory_gb = 1)

In [ ]:
# creating AKS cluster
cluster_name = 'aks-cluster-001'

if cluster_name not in ws.compute_targets:
    print(cluster_name, "does not exist. Creating a new one...")
    print('Creating provisioniong config for Aks cluster....')

    aks_config = AksCompute.provisioning_configuration(location='Canada Central',
                                                       vm_size='STANDARD_D11_V2',
                                                       agent_count=1,
                                                       cluster_purpose='DevTest')

    print("Creating the AKS Cluster...")
    production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)
    production_cluster.wait_for_completion(show_output=True)
else:
    print(cluster_name, "exists. Using it...")
    production_cluster = ws.compute_targets[cluster_name]

In [ ]:
# Deploying the model
service = Model.deploy(workspace= ws,
                       name="mlmodel", 
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           deployment_target=production_cluster,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

In [ ]:
# Preparing test data
import pandas as pd
import json
data = pd.read_csv('test.csv')
xt = data.values.tolist()
test = json.dumps({"data":xt})

In [ ]:
test

In [ ]:
# Testing 1

from azureml.core import Webservice
service = Webservice(workspace=ws, name="mldemomodel")
print(service.scoring_uri)
print(service.swagger_uri)
prediction = service.run(input_data=test)
prediction

In [ ]:
# Testing 2
import requests
headers = {'content-Type':'application/json'}

response = requests.post(service.scoring_uri, test, headers=headers)
response.text